# Advanced DataFrames Exercises Part 1

1. Run `python -m pip install pymysql` from your terminal to install the mysql client (any folder is fine)

2. cd into your exercises folder for this module and run `echo env.py >> .gitignore`
3. Create a function named `get_db_url`. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

3. Use your function to obtain a connection to the employees database. Once you have successfully run a query:
- Intentionally make a typo in the database url. What kind of error message do you see?
- Intentionally make an error in your SQL query. What does the error message look like?

- Read the employees and titles tables into two separate DataFrames.
- How many rows and columns do you have in each DataFrame? Is that what you expected?
- Display the summary statistics for each DataFrame.
- How many unique titles are in the titles DataFrame?
- What is the oldest date in the to_date column?
- What is the most recent date in the to_date column?

In [1]:
from env import get_db_url
import pandas as pd
import numpy as np

from pydataset import data

In [22]:
from env import user, password, host

In [42]:
def get_db_url(db):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [50]:
query = '''SELECT * FROM employees limit 10'''

In [51]:
pd.read_sql(query, get_db_url('employees'))

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


In [ ]:
url = get_db_url("employees")
sql = """
SELECT * FROM employees
"""

employees = pd.read_sql(sql, url)
employees.shape

In [54]:
sql = """
SELECT * FROM titles
WHERE to_date LIKE '9999-%'
LIMIT 100
"""

titles = pd.read_sql(sql, url)
titles.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Senior Engineer,1995-12-01,9999-01-01
4,10005,Senior Staff,1996-09-12,9999-01-01


In [4]:
titles.shape

(443308, 4)

In [5]:
employees.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [8]:
titles.describe(include="all")

,emp_no,title,from_date,to_date
count,443308.000000,443308,443308,443308
unique,NaN,7,6393,5888
top,NaN,Engineer,1998-10-25,9999-01-01
freq,NaN,115003,132,240124
mean,253075.034430,NaN,NaN,NaN
std,161853.292613,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN
25%,84855.750000,NaN,NaN,NaN
50%,249847.500000,NaN,NaN,NaN
75%,424891.250000,NaN,NaN,NaN


In [8]:
# How many unique titles are in the titles DataFrame?
# Example of solving the problem only with SQL
pd.read_sql("SELECT COUNT(DISTINCT title) AS unique_title_count from titles", url)

,unique_title_count
0,7


In [9]:
# How many unique titles are in the titles DataFrame?
# pandas approach
len(titles.title.unique())

7

In [10]:
# What is the oldest date in the to_date column?
pd.read_sql("SELECT MIN(to_date) FROM titles", url)

,MIN(to_date)
0,1985-03-01


In [11]:
titles.to_date.min()

datetime.date(1985, 3, 1)

In [18]:
# What is the most recent date in the to_date column?
pd.read_sql("SELECT MAX(to_date) FROM titles", url)

,MAX(to_date)
0,9999-01-01


In [13]:
# pandas approach
titles.dtypes

emp_no        int64
title        object
from_date    object
to_date      object
dtype: object

In [14]:
# pd.to_datetime(titles.to_date) this would work except for the 9999 year.

In [15]:
titles.to_date.max()

datetime.date(9999, 1, 1)

In [16]:
titles.to_date.min()

datetime.date(1985, 3, 1)

## Part 2 Exercises

Copy the users and roles DataFrames from the examples above.

What is the result of using a right join on the DataFrames?

What is the result of using an outer join on the DataFrames?

What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [19]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [20]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


#### 2. What do you think a right join would look like? 

- Jane and Mike are not included because they had NaN values in the foreign key column, `role_id`.


- The `commenter` role is included even through it is not present in the left table in the join.


- A right join uses only the keys from the right DataFrame.
    - If I set `indicator=True`, I can get a good picture of which rows are coming from where.
    - Notice that I only get back rows that are in `both` or `right_only` in a right join.

#### 3. What would an outer join look like? 
- This returns a DataFrame with all of the rows from both tables.


- Again, setting `indicator=True` allows me to see exactly what's happening here.
    - In outer join, I receive rows that are in `both`, `left_only`, and `right_only`.
    - Basically, I get all of the rows from both DataFrames.

#### 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

#### 5.  Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:

In [55]:
# Output and read the documentation for the mpg dataset.
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




#### 10. How many different manufacturers are there?

#### 11. How many different models are there?

In [176]:
mpg.model.nunique()

38

#### 12 & 13. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car. Also average_mileage

#### 14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [134]:
# I can group by my boolean column and get the average of city and highway.


#### 15. On average, which manufacturer has the best miles per gallon?

- I'll look at this question in a couple of different ways.


- The main takeaway here:  
    - we can use the `.groupby()` method on a column and perform an aggregation on another, or other, column(s).


- I can look at the highest average for `average_mileage` by `manufacturer` as a target.



#### 16. Do automatic or manual cars have better miles per gallon?

- There are so many ways I can handle this question. 
    - I'm going to create a column, `transmission_category`, that contains `auto` if a car has any type of automatic transmission 
    - and `manual` if a car has any type of manual transmission. 
    
    
- Now, I can use `transmission_category` to group my data by automatic or manual transmission.


### 3. Use your get_db_url function to help you explore the data from the chipotle database. Use the data to answer the following questions:

In [323]:
query = '''
                     SELECT *
                     FROM orders;
                     '''

#### 2. What is the total price for each order?

- `item_price` is an object, so before I can treat it like a float, I have to clean it up.

#### 3. What are the most popular 3 items?


- Basically, which three items are ordered the most.**

#### 4. Which item has produced the most revenue?


- We found that  `item_price` is really like a calulation of the quantity of an item in an order times the item's price.

#### 5. Join the employees and titles DataFrames together.

#### 6. For each title, find the hire date of the employee that was hired most recently with that title.

#### 7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)